In [20]:
!wget https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/functions.py

!wget https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/files/assets.txt
!wget https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/files/risks.txt

--2021-04-11 22:29:17--  https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6913 (6.8K) [text/plain]
Saving to: ‘functions.py.1’

functions.py.1      100%[===================>]   6.75K  --.-KB/s    in 0s      

2021-04-11 22:29:17 (63.8 MB/s) - ‘functions.py.1’ saved [6913/6913]

--2021-04-11 22:29:17--  https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/files/assets.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 395

In [21]:

import functions
from functions import *
import spacy
nlp = spacy.load('en_core_web_sm')#, disable=['parser', 'ner'])

!wget https://raw.githubusercontent.com/tylerneylon/explacy/master/explacy.py

--2021-04-11 22:29:18--  https://raw.githubusercontent.com/tylerneylon/explacy/master/explacy.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6896 (6.7K) [text/plain]
Saving to: ‘explacy.py.1’

explacy.py.1        100%[===================>]   6.73K  --.-KB/s    in 0s      

2021-04-11 22:29:19 (37.1 MB/s) - ‘explacy.py.1’ saved [6896/6896]



In [22]:
import explacy

sentence = 'Accidents, Fire, Natural Disasters, Terrorism and Outage Power expose Real Estate with probability 6 and impact 3.'
             
nlp = spacy.load('en')

# explacy.print_parse_info(nlp, sentences)


explacy.print_parse_info(nlp, sentence)

Dep tree           Token       Dep type Lemma       Part of Sp
────────────────── ─────────── ──────── ─────────── ──────────
┌─►┌───────────┬── Accidents   nsubj    accident    NOUN      
│  │           └─► ,           punct    ,           PUNCT     
│  └─►┌────────┬── Fire        conj     Fire        PROPN     
│     │        └─► ,           punct    ,           PUNCT     
│     │        ┌─► Natural     compound Natural     PROPN     
│     └─►┌─────┼── Disasters   conj     Disasters   PROPN     
│        │     └─► ,           punct    ,           PUNCT     
│        └─►┌──┬── Terrorism   conj     Terrorism   PROPN     
│           │  └─► and         cc       and         CCONJ     
│           │  ┌─► Outage      compound Outage      PROPN     
│           └─►└── Power       conj     Power       PROPN     
└────┬┬─────┬───── expose      ROOT     expose      VERB      
     ││     │  ┌─► Real        compound real        ADJ       
     ││     └─►└── Estate      dobj     Estate      PRO

In [23]:
import functions
from functions import *
import spacy
nlp = spacy.load('en_core_web_sm')#, disable=['parser', 'ner'])

assets_file = '/content/assets.txt'
assets_dict = terms_dict(assets_file)

risks_file = '/content/risks.txt'
risks_dict = terms_dict(risks_file)

In [26]:
def process_asset_risks_sentence(sentence: str, result: {}):
  """Function that process a sentence finding the terms
  risks, asset, prob and impact"""


  doc = nlp(str(sentence))

  raw_risks = extract_subject(sentence)
  risks= []
  aux = []
  for element in raw_risks:
    if element == "," or element == "and":
      risks.append(aux)
      aux = []
    elif element == raw_risks[-1]:
      risks.append(aux)
    else:
      aux.append(element)


  cont = 0
  for risk in risks:
    cont += 1
    risk = clean(risk)
    risk_matches = search_matches(risks_dict, risk)
    risk_count = counter(risk_matches)
    risk = higher_frecuency_term(risk_count)
    result["riskType"+str(cont)] = risk

  asset = extract_dobject(sentence)      
  asset = clean(asset)
  asset_matches = search_matches(assets_dict, asset)
  asset_count = counter(asset_matches)
  asset = higher_frecuency_term(asset_count)
  result["asset"] = asset 

  result["prob"] = extract_prob(sentence)
  result["impact"] = extract_impact(sentence)


In [27]:
result= {}

sentence = 'Accidents, Fire, Natural Disasters, Terrorism and Outage Power expose Real Estate with probability 6 and impact 3.'
# ojo que si ponemos power outage no funciona, si hay términos de varias palabras
# y no funciona, probar a cambiar de orden los términos


process_asset_risks_sentence(sentence, result)
result = {"result": [result]}
write_json("results.json", result) # escribir archivo con los resultados
result

{'result': [{'asset': 'RealEstate',
   'impact': 3,
   'prob': 6,
   'riskType1': 'AccidentRisk',
   'riskType2': 'FireRisk',
   'riskType3': 'NaturalDisasterRisk',
   'riskType4': 'TerrorismAttackRisk',
   'riskType5': 'PowerOutageRisk'}]}